In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random
from IPython.display import Image
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate,GridSearchCV
from surprise.prediction_algorithms import CoClustering
from surprise.prediction_algorithms import NMF
from surprise import accuracy
from collections import Counter

In [2]:
data=pd.read_csv('Online Retail.csv')

In [3]:
data.head()

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
0,536365,01-12-2010,08:26:00 AM,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,01-12-2010,08:26:00 AM,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,01-12-2010,08:26:00 AM,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,01-12-2010,08:26:00 AM,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,01-12-2010,08:26:00 AM,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [4]:
data.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
InvoiceDate         0
InvoiceTime         0
StockCode           0
Quantity            0
UnitPrice           0
Totalsale           0
Country             0
dtype: int64

In [5]:
data1 = data.dropna()
data1.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,406829.000000,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,20.401854,15287.690570
std,248.693370,69.315162,427.591718,1713.600303
min,-80995.000000,0.000000,-168469.600000,12346.000000
25%,2.000000,1.250000,4.200000,13953.000000
50%,5.000000,1.950000,11.100000,15152.000000
75%,12.000000,3.750000,19.500000,16791.000000
max,80995.000000,38970.000000,168469.600000,18287.000000


In [6]:
data1.StockCode = data1.StockCode.astype(str)

C:\Users\bismark\AppData\Local\Temp\ipykernel_19900\3251052305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.StockCode = data1.StockCode.astype(str)


In [7]:
purchase_data = (data1.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
purchase_data.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
def encode_units(x):
    if x < 1: # If the quantity is less than 1
        return 0 # Not purchased
    if x >= 1: # If the quantity is greater than 1
        return 1 # Purchased

In [10]:
purchase_data = purchase_data.applymap(encode_units)
purchase_data.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
user_similarities = cosine_similarity(purchase_data)
user_similarity_data = pd.DataFrame(user_similarities,index=purchase_data.index,columns=purchase_data.index)
user_similarity_data.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038814,0.0,0.025876,0.136641,0.094742,...,0.0,0.0,0.054656,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027995,0.0,0.027995,0.118262,0.146427,...,0.0,0.0,0.118262,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.138314,0.0,0.030737,0.032461,0.144692,...,0.0,0.0,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031846,0.0,0.000000,0.000000,0.033315,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [12]:
def fetch_similar_users(user_id,k=5):
    # separating data rows for the entered user id
    user_similarity = user_similarity_data[user_similarity_data.index ==user_id]
    # a data of all other users
    other_users_similarities = user_similarity_data[user_similarity_data.index != user_id]
    # calcuate cosine similarity between user and each other user
    similarities = cosine_similarity(user_similarity,other_users_similarities)[0].tolist()
    user_indices = other_users_similarities.index.tolist()
    index_similarity_pair = dict(zip(user_indices, similarities))
    # sort by similarity
    sorted_index_similarity_pair = sorted(index_similarity_pair.items(),reverse=True)
    top_k_users_similarities = sorted_index_similarity_pair[:k]
    similar_users = [u[0] for u in top_k_users_similarities]
    print('The users with behaviour similar to that of user {0} are:'.
    format(user_id))
    return similar_users

In [14]:
similar_users = fetch_similar_users(12347)
similar_users

The users with behaviour similar to that of user 12347 are:


[18287.0, 18283.0, 18282.0, 18281.0, 18280.0]

In [15]:
def simular_users_recommendation(userid):
    similar_users = fetch_similar_users(userid)
    #obtaining all the items bought by similar users
    simular_users_recommendation_list = []
    for j in similar_users:
        item_list = data1[data1["CustomerID"]==j]['StockCode'].to_list()
        simular_users_recommendation_list.append(item_list)
    #this gives us multi-dimensional list
    # we need to flatten it
    flat_list = []
    for sublist in simular_users_recommendation_list:
        for item in sublist:
            flat_list.append(item)
    final_recommendations_list = list(dict.fromkeys(flat_list))
    # storing 10 random recommendations in a list
    ten_random_recommendations = random.sample(final_recommendations_list, 10)
    print('Items bought by Similar users based on Cosine Similarity')
    #returning 10 random recommendations
    return ten_random_recommendations

In [16]:
simular_users_recommendation(12347)


The users with behaviour similar to that of user 12347 are:
Items bought by Similar users based on Cosine Similarity


['22725',
 '22620',
 '22418',
 '21817',
 '22386',
 '35970',
 '90099',
 '23509',
 '22358',
 '22583']

In [21]:
items_purchase_data = (data1.groupby(['StockCode','CustomerID'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('StockCode'))
items_purchase_data.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10123C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10124A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
items_purchase_data = items_purchase_df.applymap(encode_units)

In [22]:
item_similarities = cosine_similarity(items_purchase_data)

In [23]:
item_similarity_data = pd.DataFrame(item_similarities,index=items_purchase_data.index,columns=items_purchase_data.index)
item_similarity_data.head()


StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.001548,0.000990,0.0,0.000000,0.853890,0.052106,0.021922,0.004643,...,0.0,0.0,0.0,0.03875,0.0,-0.000051,0.0,0.000258,0.0,0.070843
10080,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.004958,0.020655,0.011878,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000006,0.0,0.000000
10120,0.001548,0.0,1.000000,0.004903,0.0,0.000000,0.001600,0.042560,0.010420,0.014993,...,0.0,0.0,0.0,0.00000,0.0,-0.000021,0.0,0.007154,0.0,0.000000
10123C,0.000990,0.0,0.004903,1.000000,0.0,0.000000,0.004417,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.000000,1.0,0.491784,0.001099,0.014973,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [24]:
def fetch_similar_items(item_id,k=10):
    # separating data rows of the selected item
    item_similarity = item_similarity_data[item_similarity_data.index ==item_id]
    # a data of all other items
    other_items_similarities = item_similarity_data[item_similarity_data.index != item_id]
    # calculate cosine similarity between selected item with other items
    similarities = cosine_similarity(item_similarity,other_items_similarities)[0].tolist()
    # create list of indices of these items
    item_indices = other_items_similarities.index.tolist()
    # create key/values pairs of item index and their similarity
    index_similarity_pair = dict(zip(item_indices, similarities))
    # sort by similarity
    sorted_index_similarity_pair = sorted(index_similarity_pair.items())
    # grab k items from the top
    top_k_item_similarities = sorted_index_similarity_pair[:k]
    similar_items = [u[0] for u in top_k_item_similarities]
    print('Similar items based on purchase behaviour (item-to-itemcollaborative filtering)')
    return similar_items

In [25]:
similar_items = fetch_similar_items('10002')
similar_items

Similar items based on purchase behaviour (item-to-itemcollaborative filtering)


['10080',
 '10120',
 '10123C',
 '10124A',
 '10124G',
 '10125',
 '10133',
 '10135',
 '11001',
 '15030']

In [31]:
def simular_item_recommendation(userid):
    simular_items_recommendation_list = []
    #obtaining all the similar items to items bought by user
    item_list = data1[data1["CustomerID"]==userid]['StockCode'].to_list()

    for item in item_list:
        similar_items = fetch_similar_items(item)
        simular_items_recommendation_list.append(item_list)
    #this gives us multi-dimensional list
    # we need to flatten it
    flat_list = []
    for sublist in simular_items_recommendation_list:
        for item in sublist:
            flat_list.append(item)
    final_recommendations_list = list(dict.fromkeys(flat_list))
    # storing 10 random recommendations in a list
    ten_random_recommendations = random.sample(final_recommendations_list, 10)
    print('Similar Items bought by our users based on Cosine Similarity')
    #returning 10 random recommendations
    return ten_random_recommendations

In [32]:
simular_item_recommendation(12347)

Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviour (item-to-itemcollaborative filtering)
Similar items based on purchase behaviou

['85232D',
 '20780',
 '21154',
 '22423',
 '22699',
 '22805',
 '23174',
 '84992',
 '23480',
 '47559B']

In [34]:
data3 = items_purchase_data.stack().to_frame()
#Renaming the column as Quantity
data3 = data3.reset_index().rename(columns={0:"Quantity"})
data3

,StockCode,CustomerID,Quantity
0,10002,12346.0,0.0
1,10002,12347.0,0.0
2,10002,12348.0,0.0
3,10002,12349.0,0.0
4,10002,12350.0,0.0
...,...,...,...
16106443,POST,18280.0,0.0
16106444,POST,18281.0,0.0
16106445,POST,18282.0,0.0
16106446,POST,18283.0,0.0


In [35]:
print(items_purchase_data.shape)
print(data3.shape)

(3684, 4372)
(16106448, 3)


In [36]:
customer_ids = data1['CustomerID']
item_ids = data1['StockCode']

In [37]:
count_orders = Counter(customer_ids)
customer_count_df = pd.DataFrame.from_dict(count_orders, orient='index').reset_index().rename(columns={0:"Quantity"})

In [38]:
customer_count_df = customer_count_df[customer_count_df["Quantity"]>120]
customer_count_df.rename(columns={'index':'CustomerID'},inplace=True)
customer_count_df

,CustomerID,Quantity
0,17850.0,312
1,13047.0,196
2,12583.0,251
6,14688.0,359
8,15311.0,2491
...,...,...
4289,15872.0,128
4296,13596.0,166
4309,16800.0,164
4324,13521.0,436


In [39]:
count_items = Counter(item_ids)


In [40]:
item_count_df = pd.DataFrame.from_dict(count_items, orient='index').reset_index().rename(columns={0:"Quantity"})

In [41]:
item_count_df = item_count_df[item_count_df["Quantity"]>120]
item_count_df.rename(columns={'index':'StockCode'},inplace=True)
item_count_df

,StockCode,Quantity
0,85123A,2077
1,71053,272
2,84406B,250
3,84029G,330
4,84029E,333
...,...,...
3619,23569,185
3622,23582,244
3623,23583,312
3624,23581,319


In [44]:
# counting no. of orders made by each customer
count_orders = Counter(customer_ids)
# storing the count and customer id in a dataframe
customer_count_df = pd.DataFrame.from_dict(count_orders, orient='index').reset_index().rename(columns={0:"Quantity"})

customer_count_df = customer_count_df[customer_count_df["Quantity"]>120]

customer_count_df.rename(columns={'index':'CustomerID'},inplace=True)
customer_count_df

,CustomerID,Quantity
0,17850.0,312
1,13047.0,196
2,12583.0,251
6,14688.0,359
8,15311.0,2491
...,...,...
4289,15872.0,128
4296,13596.0,166
4309,16800.0,164
4324,13521.0,436


In [45]:
count_items = Counter(item_ids)


# storing the count and item description in a dataframe
item_count_df = pd.DataFrame.from_dict(count_items, orient='index').reset_index().rename(columns={0:"Quantity"})

item_count_df = item_count_df[item_count_df["Quantity"]>120]

item_count_df.rename(columns={'index':'StockCode'},inplace=True)
item_count_df

,StockCode,Quantity
0,85123A,2077
1,71053,272
2,84406B,250
3,84029G,330
4,84029E,333
...,...,...
3619,23569,185
3622,23582,244
3623,23583,312
3624,23581,319


In [46]:
#Merging stacked df with item count df
data4 = pd.merge(data3, item_count_df, on='StockCode', how='inner')
#Merging with customer count df
data4 = pd.merge(data4, customer_count_df, on='CustomerID', how='inner')
# dropping columns which are not necessary
data4.drop(['Quantity_y','Quantity_x'],axis=1,inplace=True)
data4

,StockCode,CustomerID,Quantity
0,10133,12347.0,182
1,10135,12347.0,182
2,15036,12347.0,182
3,15056BL,12347.0,182
4,15056N,12347.0,182
...,...,...,...
938837,85199S,18283.0,756
938838,85227,18283.0,756
938839,C2,18283.0,756
938840,M,18283.0,756


In [47]:
reader = Reader(rating_scale=(0,5095))
formated_data = Dataset.load_from_df(data4, reader)
train_set, test_set = train_test_split(formated_data, test_size= 0.2)


In [48]:
algo1 = NMF()

algo1.fit(train_set)
# model prediction
pred1 = algo1.test(test_set)

# RMSE
accuracy.rmse(pred1)
#MAE
accuracy.mae(pred1)

RMSE: 521.9491
MAE:  295.9491


295.9491455492547

In [49]:
cross_validate(algo1, formated_data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    530.7587536.4238524.9397526.6436539.3872531.63065.5414  
MAE (testset)     296.1018296.9337295.2527295.7987298.8771296.59281.2650  
Fit time          12.46   12.80   12.69   12.77   12.70   12.68   0.12    
Test time         1.35    1.19    1.06    1.36    1.05    1.20    0.13    


{'test_rmse': array([530.75869317, 536.42382038, 524.93972269, 526.6435715 ,
        539.38717259]),
 'test_mae': array([296.10179985, 296.9337435 , 295.25271209, 295.79869766,
        298.87708453]),
 'fit_time': (12.45947527885437,
  12.8003568649292,
  12.68622612953186,
  12.769476890563965,
  12.703020334243774),
 'test_time': (1.352189540863037,
  1.1914443969726562,
  1.0617170333862305,
  1.3566255569458008,
  1.0516929626464844)}

In [51]:
data1[(data1['StockCode']=='47590B')&(data1['CustomerID']==15738)].Quantity.sum()

99

In [52]:
algo1.test([['47590B',15738,78]])

[Prediction(uid='47590B', iid=15738, r_ui=78, est=3.5285548387949603, details={'was_impossible': False})]